In [1]:
%matplotlib inline

In [10]:
from tqdm.notebook import tqdm
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import ContinuousSpace
from mesa.batchrunner import BatchRunner
from mesa.visualization.modules import CanvasGrid
from mesa.datacollection import DataCollector
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Line
from bokeh.models.annotations import Legend
from bokeh.palettes import Category10
import panel as pn
import math
pn.extension()

# Mesa
> Mesa is a modular framework for building, analyzing and visualizing **agent-based models**.

[Documentation](https://mesa.readthedocs.io/en/master/overview.html)

# Introduction

The simulation is composed of the following components:
- `Model` class - representing the model.
- `Agent` class - representing the agent.
- `Scheduler` class - agent activation strategy
- `step` method - simulation logic

In the simulation we will have a continouous space. Agents will move within it and infect other agents.

# Simulation model

In [3]:
def compute_alive(model):
    return # TODO: return number of alive agents

In [3]:
def compute_alive(model):
    return model.num_agents

In [3]:
def compute_sick(model):
    return # TODO: return number of sick agents

In [3]:
def compute_sick(model):
    return len([a for a in model.schedule.agents if a.is_sick])

In [ ]:
def change_position(pos, speed, heading):
    return # TODO: return new position

In [26]:
def change_position(pos, speed, heading):
    x, y = pos
    return (
        x + speed * math.sin(heading),
        y + speed * math.cos(heading)
    )

In [30]:
class EpidemicAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model, is_sick):
        super().__init__(unique_id, model)
        self.is_sick = is_sick
        self.infection_probability = 0.3
        self.infection_radius = 30
        self.speed = 10
        self.heading = self.random.uniform(0, 2 * math.pi)
        self.time_infected = 0

    def move(self):
        # Move agent
        new_pos = change_position(self.pos, self.speed, self.heading)
        self.model.grid.move_agent(self, new_pos)
        # Update velocity
        self.speed += self.random.uniform(-3, 3)
        self.heading += self.random.uniform(-math.pi/4, math.pi/4)
    
    def infect(self):
        # TODO: Select agents within `infection_radius`
        # TODO: Infect others with `infection_probability`
        
        # DONE:
        # Select agents within radius
        mates = self.model.grid.get_neighbors(self.pos, self.infection_radius, include_center=False)
        for other in mates:
            # Infect agent with probability
            if self.random.random() < self.infection_probability:
                other.become_infected()

    def become_infected(self):
        if not self.is_sick:
            self.is_sick = True
            self.time_infected = 0
    
    def try_die(self):
        pass

    def step(self):
        # TODO: Move agent, then infect if is sick
        # DONE:
        self.move()
        if self.is_sick:
            self.time_infected += 1
            self.infect()

In [31]:
class EpidemicModel(Model):
    """A model with some number of agents."""
    def __init__(self, N, x_max, y_max):
        self.num_agents = N
        self.grid = ContinuousSpace(x_max, y_max, torus=True)
        self.schedule = RandomActivation(self)

        # Create agents
        for i in range(self.num_agents):
            is_sick = self.random.random() < 0.1
            a = EpidemicAgent(i, self, is_sick=is_sick)
            self.schedule.add(a)
            
            # TODO: Add agent to a random place on the grid
            # TIP: self.random contains initialized random object
            # TIP: grid has width and height attributes
            # TIP: grid has place_agent method
            
            # DONE:
            # Add the agent to a random grid cell
            x = self.random.uniform(0, self.grid.width)
            y = self.random.uniform(0, self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.datacollector = DataCollector(
            model_reporters = {
                'alive': compute_alive,
                'sick': compute_sick
            },
            agent_reporters = {
                'is_sick': 'is_sick',
                'speed': 'speed'
            }
        )
        
    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [ ]:
model = # TODO: Initialize model
for _ in tqdm(range(100)):
    # TODO: Run model step

In [33]:
# DONE
model = EpidemicModel(100, 1000, 1000)
for _ in tqdm(range(100)):
    model.step()

In [34]:
df = model.datacollector.get_model_vars_dataframe()

In [35]:
def plot_states_bokeh(model, iteration):
    X = model.datacollector.get_model_vars_dataframe()
    source = ColumnDataSource(X)
    colors = Category10[10]
    items = []
    p = figure(plot_width=600, plot_height=400, tools=[], title=f'step: {iteration}', x_range=(0,100))        
    for i, c in enumerate(X.columns):
        line = Line(x='index', y=c, line_color=colors[i], line_width=3, line_alpha=.8, name=c)
        glyph = p.add_glyph(source, line)
        i+=1
        items.append((c, [glyph]))

    p.xaxis.axis_label = 'Step'
    p.add_layout(Legend(location='center_right', items=items))
    p.background_fill_color = "#ffffff"
    p.background_fill_alpha = 1
    p.legend.label_text_font_size = "10pt"
    p.title.text_font_size = "15pt"
    p.toolbar.logo = None
    p.sizing_mode = 'scale_height'    
    return p

In [36]:
def plot_agents_bokeh(model):
    def plot_agents(p, agents, color):
        xx = list(map(lambda x: x.pos[0], agents))
        yy = list(map(lambda x: x.pos[1], agents))
        ss = list(map(lambda x: x.infection_radius, agents))
        p.circle_dot(x=xx, y=yy, size=ss, alpha=0.5, fill_color=color)
    
    agents = model.schedule.agents
    w = model.grid.width
    h = model.grid.height
    p = figure(plot_width=500, plot_height=500, x_range=(0, w), y_range=(0, h))
    
    plot_agents(p, list(filter(lambda x: x.is_sick, agents)), "red")
    plot_agents(p, list(filter(lambda x: not x.is_sick, agents)), "blue")
    
    p.grid.grid_line_color = None    
    p.axis.axis_line_color = None
    p.toolbar.logo = None
    return p

## Visualization

In [37]:
plot_pane = pn.pane.Bokeh()
grid_pane = pn.pane.Bokeh()
pn.Row(plot_pane, grid_pane, sizing_mode='stretch_width')

Row(sizing_mode='stretch_width')
    [0] Bokeh(None)
    [1] Bokeh(None)

## Run simulation

In [38]:
model = EpidemicModel(100, 1000, 1000)
for i in range(100):
    model.step()
    plot_pane.object = plot_states_bokeh(model, i)
    grid_pane.object = plot_agents_bokeh(model)

## TODO
- Add agent's recovery or death after some number of iterations (use `time_infected` parameter)
- Add death/recovered metric

# Batch running